In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose, path):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword='+keyword+'&sc.keyword='+keyword+'&locT=&locId=&jobType='
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(50)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            driver.execute_script("arguments[0].click();", job_button)  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
#This line will open a new chrome window and start the scraping
path = "/Users/lenovo/Desktop/data_science/ds_salary_proj/chromedriver"
df = get_jobs("data scientist", 500, False, path)
df

Progress: 0/500
Progress: 1/500
Progress: 2/500
Progress: 3/500
Progress: 4/500
Progress: 5/500
Progress: 6/500
Progress: 7/500
Progress: 8/500
Progress: 9/500
Progress: 10/500
Progress: 11/500
Progress: 12/500
Progress: 13/500
Progress: 14/500
Progress: 15/500
Progress: 16/500
Progress: 17/500
Progress: 18/500
Progress: 19/500
Progress: 20/500
Progress: 21/500
Progress: 22/500
Progress: 23/500
Progress: 24/500
Progress: 25/500
Progress: 26/500
Progress: 27/500
Progress: 28/500
Progress: 29/500
Progress: 30/500
Progress: 31/500
Progress: 32/500
Progress: 33/500
Progress: 34/500
Progress: 35/500
Progress: 36/500
Progress: 37/500
Progress: 38/500
Progress: 39/500
Progress: 40/500
Progress: 41/500
Progress: 42/500
Progress: 43/500
Progress: 44/500
Progress: 45/500
Progress: 46/500
Progress: 47/500
Progress: 48/500
Progress: 49/500
Progress: 50/500
Progress: 51/500
Progress: 52/500
Progress: 53/500
Progress: 54/500
Progress: 55/500
Progress: 56/500
Progress: 57/500
Progress: 58/500
Progres

Progress: 462/500
Progress: 463/500
Progress: 464/500
Progress: 465/500
Progress: 466/500
Progress: 467/500
Progress: 468/500
Progress: 469/500
Progress: 470/500
Progress: 471/500
Progress: 472/500
Progress: 473/500
Progress: 474/500
Progress: 475/500
Progress: 476/500
Progress: 477/500
Progress: 478/500
Progress: 479/500
Progress: 480/500
Progress: 481/500
Progress: 482/500
Progress: 483/500
Progress: 484/500
Progress: 485/500
Progress: 486/500
Progress: 487/500
Progress: 488/500
Progress: 489/500
Progress: 490/500
Progress: 491/500
Progress: 492/500
Progress: 493/500
Progress: 494/500
Progress: 495/500
Progress: 496/500
Progress: 497/500
Progress: 498/500
Progress: 499/500
Progress: 500/500


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,Job Description:\n\n\nJob Title - Data Scienti...,3.5,Deutsche Bank\n3.5,Pune,"Frankfurt am Main, Germany",10000+ employees,1870,Company - Public,Banks & Building Societies,Finance,₹500+ billion (INR),-1
1,Data Scientist,-1,If you are a data junkie who would like to wra...,3.0,BlueJeans\n3.0,Bengaluru,"San Jose, CA",201 to 500 employees,2009,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
2,"Consultant, Data Science and Analytics",-1,"What We'll Bring:\n\nAt TransUnion, we have a ...",4.0,TransUnion\n4.0,Chennai,"Chicago, IL",5001 to 10000 employees,1968,Company - Public,Financial Analytics & Research,Finance,₹100 to ₹500 billion (INR),-1
3,Data Engineer,-1,About phData\n\nWe build next-generation strat...,2.9,phData\n2.9,Bengaluru,"Minneapolis, MN",51 to 200 employees,2014,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,-1
4,Data Scientist,-1,Overview\n\n\nWe have an exciting opportunity ...,3.8,Quanticate\n3.8,Bengaluru,"Hitchin, United Kingdom",201 to 500 employees,1995,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹1 to ₹5 billion (INR),"GCE Solutions, Parexel, IQVIA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Quantitative Analyst,-1,Morningstar Indexes- http://indexes.morningsta...,4.1,Morningstar\n4.1,Mumbai,"Chicago, IL",5001 to 10000 employees,1984,Company - Public,Financial Analytics & Research,Finance,₹100 to ₹500 billion (INR),"Thomson Reuters, FactSet, Bloomberg L.P."
496,Sr Data Analyst,-1,We collect peta-bytes of retail data and are l...,3.5,CodeHall\n3.5,Bengaluru,"Bengaluru, India",1 to 50 employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable,-1
497,Clinical Data Analyst,-1,Category: Executive\n\nLocation: Chennai and B...,3.5,Navitas Life Sciences\n3.5,Bengaluru,"Princeton, NJ",501 to 1000 employees,-1,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,-1
498,Data Science Engineer,-1,Experience : 2 to 4 years\nLocation : Mumbai\n...,-1,Mystro,Mumbai,"San Francisco, CA",1 to 50 employees,-1,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,-1
